In [128]:
# -*- coding: utf-8 -*-
# 基本モジュール
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm

import time
import datetime
import dateutil.parser #変数の時間型への変換で使用
from dateutil.relativedelta import relativedelta

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
np.random.seed(0)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import matplotlib.pylab as plt
from matplotlib.font_manager import FontProperties
fp = FontProperties(fname='meiryo.ttc')
%matplotlib inline

In [2]:
in_data=pd.read_csv("data.csv")

In [3]:
in_data.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5


In [4]:
in_data.columns

Index(['action_type', 'combined_shot_type', 'game_event_id', 'game_id', 'lat',
       'loc_x', 'loc_y', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'season', 'seconds_remaining', 'shot_distance', 'shot_made_flag',
       'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
       'team_id', 'team_name', 'game_date', 'matchup', 'opponent', 'shot_id'],
      dtype='object')

In [6]:
for tmp_col in in_data.columns:
    print(tmp_col+":")
    print(in_data[tmp_col].value_counts())
    print("")

action_type:
Jump Shot                             18880
Layup Shot                             2567
Driving Layup Shot                     1978
Turnaround Jump Shot                   1057
Fadeaway Jump Shot                     1048
Running Jump Shot                       926
Pullup Jump shot                        476
Turnaround Fadeaway shot                439
Slam Dunk Shot                          411
Reverse Layup Shot                      395
Jump Bank Shot                          333
Driving Dunk Shot                       310
Dunk Shot                               262
Tip Shot                                182
Alley Oop Dunk Shot                     122
Step Back Jump shot                     118
Floating Jump shot                      114
Driving Reverse Layup Shot               97
Hook Shot                                84
Driving Finger Roll Shot                 82
Alley Oop Layup shot                     80
Reverse Dunk Shot                        75
Running Layup Shot 

Name: lon, Length: 489, dtype: int64

minutes_remaining:
0     3866
4     2855
3     2839
2     2834
1     2734
5     2651
6     2383
7     2215
9     2202
8     2139
10    2122
11    1857
Name: minutes_remaining, dtype: int64

period:
3    8296
1    8048
4    7260
2    6718
5     330
6      38
7       7
Name: period, dtype: int64

playoffs:
0    26198
1     4499
Name: playoffs, dtype: int64

season:
2005-06    2318
2008-09    2242
2002-03    2241
2007-08    2153
2009-10    2080
2001-02    2028
2006-07    1887
2000-01    1868
2010-11    1825
2003-04    1638
2011-12    1637
2012-13    1593
1999-00    1577
2004-05    1324
2015-16    1113
1997-98     989
1998-99     921
2014-15     713
1996-97     477
2013-14      73
Name: season, dtype: int64

seconds_remaining:
0     985
1     659
2     598
32    557
4     557
28    550
5     549
36    546
41    541
25    536
40    527
13    526
44    526
38    526
11    526
37    523
23    521
3     520
33    519
24    518
27    512
8     511
16    510

In [9]:
for tmp_type in np.unique(in_data["combined_shot_type"]):
    tmp_data=in_data[in_data["combined_shot_type"]==tmp_type].copy()
    print(tmp_type+":")
    print(len(tmp_data[tmp_data["shot_made_flag"]==1]))
    print(len(tmp_data[tmp_data["shot_made_flag"]==0]))

Bank Shot:
95
25
Dunk:
980
76
Hook Shot:
68
59
Jump Shot:
7708
12002
Layup:
2561
1971
Tip Shot:
53
99


In [85]:
in_data2=in_data.copy()

In [86]:
del in_data2["action_type"]

In [87]:
tmp_data=pd.get_dummies(in_data2["combined_shot_type"])
tmp_data.columns=["1_"+tmp_col for tmp_col in tmp_data]

In [88]:
in_data2=pd.concat([in_data2,tmp_data],axis=1)

In [89]:
del in_data2["game_event_id"]

In [90]:
del in_data2["game_id"]

In [91]:
del in_data2['lat']
del in_data2['lon']

In [92]:
in_data2[['minutes_remaining','seconds_remaining']]

,minutes_remaining,seconds_remaining
0,10,27
1,10,22
2,7,45
3,6,52
4,6,19
5,9,32
6,8,52
7,8,5
8,6,12
9,3,36


In [93]:
in_data2["time_remaining"]=in_data2['minutes_remaining']*60+in_data2['seconds_remaining']

In [94]:
del in_data2['minutes_remaining']
del in_data2['seconds_remaining']

In [95]:
del in_data2["combined_shot_type"]

In [96]:
def ret4(x):
    return int(str(x)[0:4])

In [97]:
in_data2["season"]=in_data2["season"].apply(ret4)

In [98]:
tmp_data=pd.get_dummies(in_data2["shot_type"])
tmp_data.columns=["15_"+tmp_col for tmp_col in tmp_data.columns]

In [99]:
in_data2=pd.concat([in_data2,tmp_data],axis=1)
del in_data2["shot_type"]

In [100]:
in_data2["shot_zone_area"].value_counts()

Center(C)                13455
Right Side Center(RC)     4776
Right Side(R)             4588
Left Side Center(LC)      4044
Left Side(L)              3751
Back Court(BC)              83
Name: shot_zone_area, dtype: int64

In [101]:
tmp_data=pd.get_dummies(in_data2["shot_zone_area"])
tmp_data.columns=["16_"+tmp_col for tmp_col in tmp_data.columns]
in_data2=pd.concat([in_data2,tmp_data],axis=1)
del in_data2["shot_zone_area"]

In [102]:
in_data2["shot_zone_basic"].value_counts()

Mid-Range                12625
Restricted Area           7136
Above the Break 3         5620
In The Paint (Non-RA)     4578
Right Corner 3             387
Left Corner 3              280
Backcourt                   71
Name: shot_zone_basic, dtype: int64

In [103]:
tmp_data=pd.get_dummies(in_data2["shot_zone_basic"])
tmp_data.columns=["17_"+tmp_col for tmp_col in tmp_data.columns]
in_data2=pd.concat([in_data2,tmp_data],axis=1)
del in_data2["shot_zone_basic"]

In [104]:
in_data2["shot_zone_range"].value_counts()

Less Than 8 ft.    9398
16-24 ft.          8315
8-16 ft.           6626
24+ ft.            6275
Back Court Shot      83
Name: shot_zone_range, dtype: int64

In [105]:
tmp_data=pd.get_dummies(in_data2["shot_zone_range"])
tmp_data.columns=["18_"+tmp_col for tmp_col in tmp_data.columns]
in_data2=pd.concat([in_data2,tmp_data],axis=1)
del in_data2["shot_zone_range"]

In [106]:
in_data2["team_id"].value_counts()

1610612747    30697
Name: team_id, dtype: int64

In [107]:
del in_data2["team_id"]

In [108]:
in_data2["team_name"].value_counts()

Los Angeles Lakers    30697
Name: team_name, dtype: int64

In [109]:
del in_data2["team_name"]

In [110]:
in_data2["game_date"].head()

0    2000-10-31
1    2000-10-31
2    2000-10-31
3    2000-10-31
4    2000-10-31
Name: game_date, dtype: object

# Unix時間を足したい

In [111]:
def ret_month(x):
    return str(x)[5:7]

In [112]:
in_data2["game_month"]=in_data2["game_date"].apply(ret_month)

In [113]:
tmp_data=pd.get_dummies(in_data2["game_month"])
tmp_data.columns=["18_gage_month_"+tmp_col for tmp_col in tmp_data.columns]
in_data2=pd.concat([in_data2,tmp_data],axis=1)
del in_data2["game_month"]
del in_data2["game_date"]

In [114]:
def isHome(x):
    if str(x)[4:5]=="@":
        ret=0
    elif str(x)[4:5]=="v":
        ret=1
    else:
        ret=99999999
    return ret

In [115]:
in_data2["home_game"]=in_data2["matchup"].apply(isHome)

In [116]:
del in_data2["matchup"]

In [117]:
in_data2["opponent"].value_counts()

SAS    1978
PHX    1781
HOU    1666
SAC    1643
DEN    1642
POR    1539
UTA    1490
MIN    1474
GSW    1356
LAC    1285
DAL    1199
MEM    1030
BOS     946
SEA     828
IND     761
PHI     720
ORL     719
DET     715
OKC     677
TOR     664
NYK     657
MIA     627
CHA     620
CLE     619
CHI     610
WAS     600
MIL     586
NOH     581
NJN     520
ATL     519
NOP     344
VAN     246
BKN      55
Name: opponent, dtype: int64

In [118]:
tmp_data=pd.get_dummies(in_data2["opponent"])
tmp_data.columns=["20_opponent_"+tmp_col for tmp_col in tmp_data.columns]
in_data2=pd.concat([in_data2,tmp_data],axis=1)
del in_data2["opponent"]

In [119]:
in_data2

,loc_x,loc_y,period,playoffs,season,shot_distance,shot_made_flag,shot_id,1_Bank Shot,1_Dunk,...,20_opponent_PHI,20_opponent_PHX,20_opponent_POR,20_opponent_SAC,20_opponent_SAS,20_opponent_SEA,20_opponent_TOR,20_opponent_UTA,20_opponent_VAN,20_opponent_WAS
0,167,72,1,0,2000,18,NaN,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,-157,0,1,0,2000,15,0.0,2,0,0,...,0,0,1,0,0,0,0,0,0,0
2,-101,135,1,0,2000,16,1.0,3,0,0,...,0,0,1,0,0,0,0,0,0,0
3,138,175,1,0,2000,22,0.0,4,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,2,0,2000,0,1.0,5,0,1,...,0,0,1,0,0,0,0,0,0,0
5,-145,-11,3,0,2000,14,0.0,6,0,0,...,0,0,1,0,0,0,0,0,0,0
6,0,0,3,0,2000,0,1.0,7,0,0,...,0,0,1,0,0,0,0,0,0,0
7,1,28,3,0,2000,2,NaN,8,0,0,...,0,0,1,0,0,0,0,0,0,0
8,-65,108,3,0,2000,12,1.0,9,0,0,...,0,0,1,0,0,0,0,0,0,0
9,-33,125,3,0,2000,12,0.0,10,0,0,...,0,0,1,0,0,0,0,0,0,0


In [122]:
train_data=in_data2[in_data2["shot_made_flag"]==in_data2["shot_made_flag"]]

In [124]:
train_data_x=train_data.copy()

In [125]:
del train_data_x["shot_made_flag"]
del train_data_x["shot_id"]

In [126]:
train_data_y=train_data["shot_made_flag"]

In [139]:
model = RandomForestClassifier(random_state=0,n_estimators=100)

In [140]:
model.fit(train_data_x, train_data_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [176]:
ypred=model.predict(train_data_x)
accuracy_score(train_data_y, ypred)

1.0

In [158]:
imp_data=pd.DataFrame([train_data_x.columns,model.feature_importances_]).T
imp_data.columns=["col","imp"]

In [166]:
imp_data=imp_data.sort_values("imp",ascending=False)
imp_data=imp_data.reset_index(drop=True)

In [169]:
imp_data["cumsum"]=imp_data.cumsum()["imp"]

In [174]:
imp_cols=np.array(imp_data[imp_data["cumsum"]<0.95]["col"])

In [175]:
model2 = RandomForestClassifier(random_state=0,n_estimators=100)
model2.fit(train_data_x[imp_cols], train_data_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [178]:
ypred=model2.predict(train_data_x[imp_cols])
accuracy_score(train_data_y, ypred)

1.0

In [143]:
eval_data=in_data2[in_data2["shot_made_flag"]!=in_data2["shot_made_flag"]].copy()

In [144]:
eval_data_x=eval_data.copy()
del eval_data_x["shot_made_flag"]
del eval_data_x["shot_id"]

In [145]:
ypred=model.predict_proba(eval_data_x)

In [179]:
ypred2=model2.predict_proba(eval_data_x[imp_cols])

In [180]:
ypred2

array([[0.68, 0.32],
       [0.49, 0.51],
       [0.59, 0.41],
       ...,
       [0.76, 0.24],
       [0.66, 0.34],
       [0.45, 0.55]])

In [152]:
submit_data=pd.concat([eval_data.reset_index(drop=True)["shot_id"],pd.DataFrame(ypred[:,1])],axis=1)
submit_data.columns=["shot_id","shot_made_flag"]
submit_data.to_csv("submit_data_0527_01.csv",index=False,header=True)

In [181]:
submit_data=pd.concat([eval_data.reset_index(drop=True)["shot_id"],pd.DataFrame(ypred2[:,1])],axis=1)
submit_data.columns=["shot_id","shot_made_flag"]
submit_data.to_csv("submit_data_0527_02.csv",index=False,header=True)